In [7]:
import numpy as np

# Historical data: Implemented story points and capacity (as presence days) for each developer from previous PIs
historical_data = {
    "Thomas": {"capacity": [16.64, 16.64], "story_points": [18, 26]},
    "Paul": {"capacity": [20.48, 20.48], "story_points": [13, 15]},
    "Yasmin": {"capacity": [17.92, 17.92], "story_points": [12, 12]},
    "Mohamed": {"capacity": [12.8, 12.8], "story_points": [28, 28]}
}

# The capacity for the next PI for each developer (in man-days)
next_PI_capacity = {
    "Thomas": 19.2,  # e.g., Thomas is expected to be available for 18 man-days in the next PI
    "Paul": 22.4,
    "Mohamed": 8.96,
    "Yasmin": 22.4
}

# Number of simulations
n_simulations = 1000

def simulate_story_points(data, next_PI_capacity, n_simulations):
    developer_story_points = {dev: [] for dev in data}
    total_story_points = []

    for _ in range(n_simulations):
        sprint_story_points = 0
        devs_story_points = {}

        for dev, records in data.items():
            # Calculate the average story points per capacity day for each developer based on historical data
            ratio = np.mean(np.array(records["story_points"]) / np.array(records["capacity"]))
            
            # Use the provided capacity for the next PI to estimate future presence
            future_presence = next_PI_capacity[dev]
            
            # Predict future story points based on the provided future capacity
            future_story_points = future_presence * ratio

            devs_story_points[dev] = future_story_points
            sprint_story_points += future_story_points

        for dev, points in devs_story_points.items():
            developer_story_points[dev].append(points)

        total_story_points.append(sprint_story_points)

    return developer_story_points, total_story_points

# Run the simulation
simulated_developer_story_points, simulated_total_story_points = simulate_story_points(historical_data, next_PI_capacity, n_simulations)

# Calculate and print the simulation results for each developer
for dev, points in simulated_developer_story_points.items():
    print(f"{dev} - Mean Predicted Story Points: {np.mean(points)}")
    print(f"{dev} - Standard Deviation: {np.std(points)}")
    print(f"{dev} - Median: {np.median(points)}")
    print(f"{dev} - 90th Percentile: {np.percentile(points, 90)}\n")

# Calculate and print the simulation results for the team
print("Total Team Predictions")
print(f"Mean Predicted Story Points: {np.mean(simulated_total_story_points)}")
print(f"Standard Deviation: {np.std(simulated_total_story_points)}")
print(f"Median: {np.median(simulated_total_story_points)}")
print(f"90th Percentile: {np.percentile(simulated_total_story_points, 90)}")


Thomas - Mean Predicted Story Points: 25.384615384615376
Thomas - Standard Deviation: 7.105427357601002e-15
Thomas - Median: 25.384615384615383
Thomas - 90th Percentile: 25.384615384615383

Paul - Mean Predicted Story Points: 15.312499999999998
Paul - Standard Deviation: 0.0
Paul - Median: 15.312499999999998
Paul - 90th Percentile: 15.312499999999998

Yasmin - Mean Predicted Story Points: 14.999999999999998
Yasmin - Standard Deviation: 0.0
Yasmin - Median: 14.999999999999998
Yasmin - 90th Percentile: 14.999999999999998

Mohamed - Mean Predicted Story Points: 19.6
Mohamed - Standard Deviation: 0.0
Mohamed - Median: 19.6
Mohamed - 90th Percentile: 19.6

Total Team Predictions
Mean Predicted Story Points: 75.29711538461538
Standard Deviation: 0.0
Median: 75.29711538461538
90th Percentile: 75.29711538461538


In [1]:
import streamlit as st
import pandas as pd
import numpy as np

# Function to simulate story points based on input data and capacity
def simulate_story_points(data, next_PI_capacity, n_simulations=1000):
    developer_story_points = {dev: [] for dev in data}
    total_story_points = []

    for _ in range(n_simulations):
        sprint_story_points = 0

        for dev, records in data.items():
            ratio = np.mean(np.array(records["story_points"]) / np.array(records["capacity"]))
            future_presence = next_PI_capacity[dev]
            future_story_points = future_presence * ratio
            developer_story_points[dev].append(future_story_points)
            sprint_story_points += future_story_points

        total_story_points.append(sprint_story_points)

    return developer_story_points, np.mean(total_story_points)

# Streamlit app title
st.title('Sprint Capacity Planner')

# Number of developers
num_devs = st.number_input('Number of developers', min_value=1, value=4)

# Input for developers' data
dev_data = {}
next_PI_capacity = {}
for i in range(1, num_devs + 1):
    st.subheader(f'Developer {i}')
    name = st.text_input(f'Developer {i} name', value=f'Dev{i}')
    capacity = st.text_input(f'Historical capacities for {name} (comma-separated)', value='15,20')
    story_points = st.text_input(f'Historical story points for {name} (comma-separated)', value='12,18')
    next_capacity = st.number_input(f'Next PI capacity for {name} (man-days)', value=15)
    
    # Parsing the input data
    capacity = list(map(float, capacity.split(',')))
    story_points = list(map(int, story_points.split(',')))

    dev_data[name] = {"capacity": capacity, "story_points": story_points}
    next_PI_capacity[name] = next_capacity

# Run simulation button
if st.button('Calculate Capacity'):
    dev_story_points, total_capacity = simulate_story_points(dev_data, next_PI_capacity)
    st.subheader('Predicted story points per developer')
    for dev, points in dev_story_points.items():
        st.text(f'{dev}: {np.mean(points):.2f} story points')

    st.subheader('Total team capacity for next PI')
    st.text(f'{total_capacity:.2f} story points')


2023-10-24 11:24:59.537 
  command:

    streamlit run /Users/taacapag/miniconda3/envs/ds_env/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
